# Install additional Python libraries

In [1]:
# !pip install quantities

# Delete existing DB

In [2]:
!rm simoc_server/db.sqlite

# Import Python libraries

In [3]:
import json
from random import randint

In [4]:
import pprint
pp = pprint.PrettyPrinter()

In [5]:
from simoc_server import app, db
from simoc_server.database.db_model import User
from simoc_server.database.seed_data import seed
from simoc_server.game_runner import GameRunnerManager, GameRunnerInitializationParams

Unknown DB_TYPE variable: "None"
Using SQLite by default


# Load Agent model

In [6]:
db.drop_all()
db.create_all()
seed.seed('agent_desc.json')

# Create new user

In [7]:
username = 'user1'
password = 'user1'

user = User.query.filter_by(username=username).first()
if not user:
    print('Creating new user')
    user = User(username=username)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()

Creating new user


# Initialize the ABM

In [8]:
game_config = {"agents": {
    "human_agent": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "food_storage": [1]}, "amount": 20}],
    "cabbage": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "lettuce": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "greenhouse_medium": [{"connections": {"power_storage": [1]}, "amount": 1}],
    "solar_pv_array": [{"connections": {"power_storage": [1]}, "amount": 100}],
    "multifiltration_purifier_post_treament": [{"connections": {"water_storage": [1, 2]}, "amount": 1}],
    "urine_recycling_processor_VCD": [{"connections": {"power_storage": [1], "water_storage": [1, 2]}, "amount": 1}]},
"storages": {
    "air_storage": [{"id": 1, "atmo_h2o": 100, "atmo_o2": 100, "atmo_co2": 100}],
    "water_storage": [{"id": 1, "h2o_potb": 100, "h2o_tret": 100}, {"id": 2, "h2o_wste": 100, "h2o_urin": 100}],
    "nutrient_storage": [{"id": 1, "sold_n": 100, "sold_p": 100, "sold_k": 100}],
    "power_storage": [{"id": 1, "enrg_kwh": 1000}],
    "food_storage": [{"id": 1, "food_edbl": 100}]},
"termination": {
    "time": {"value": 2000, "unit": "hour"}},
    "food_leaf": {"value": 10000, "unit": "kg"},
    "evacuation": {}
}

game_runner_manager = GameRunnerManager()
game_runner_init_params = GameRunnerInitializationParams(game_config)
game_runner_manager.new_game(user, game_runner_init_params)
game_runner = game_runner_manager.get_game_runner(user)

In [9]:
game_runner.agent_model.get_total_storages()

{'total_currencies': {'atmo_o2': {'value': 100,
   'capacity': 400,
   'units': 'kg'},
  'atmo_co2': {'value': 100, 'capacity': 200, 'units': 'kg'},
  'atmo_n2': {'value': 0, 'capacity': 400, 'units': 'kg'},
  'atmo_ch4': {'value': 0, 'capacity': 200, 'units': 'kg'},
  'atmo_h2': {'value': 0, 'capacity': 100, 'units': 'kg'},
  'atmo_h2o': {'value': 100, 'capacity': 200, 'units': 'kg'},
  'h2o_potb': {'value': 100, 'capacity': 800, 'units': 'kg'},
  'h2o_urin': {'value': 100, 'capacity': 400, 'units': 'kg'},
  'h2o_wste': {'value': 100, 'capacity': 500, 'units': 'kg'},
  'h2o_tret': {'value': 100, 'capacity': 500, 'units': 'kg'},
  'sold_n': {'value': 100, 'capacity': 100, 'units': 'kg'},
  'sold_p': {'value': 100, 'capacity': 100, 'units': 'kg'},
  'sold_k': {'value': 100, 'capacity': 100, 'units': 'kg'},
  'enrg_kwh': {'value': 1000, 'capacity': 1000, 'units': 'kWh'},
  'food_edbl': {'value': 100, 'capacity': 1000, 'units': 'kg'}},
 'total_storages': {'air_storage': 1,
  'water_storag

In [10]:
game_runner.agent_model.get_total_agents()

{'total_production': {'atmo_co2': {'value': 1.2800000000000007, 'units': 'kg'},
  'atmo_h2o': {'value': 2.22, 'units': 'kg'},
  'h2o_wste': {'value': 1.252499999999999, 'units': 'kg'},
  'h2o_urin': {'value': 1.25, 'units': 'kg'},
  'atmo_o2': {'value': 5.739060121999999, 'units': 'g'},
  'food_edbl': {'value': 0.0, 'units': 'g'},
  'enrg_kwh': {'value': 30.00000000000005, 'units': 'kWh'},
  'h2o_potb': {'value': 4.75, 'units': 'kg'},
  'h2o_tret': {'value': 1.36, 'units': 'kg'}},
 'total_consumption': {'atmo_o2': {'value': 984.0000000000003, 'units': 'g'},
  'h2o_potb': {'value': 8.488623750000034, 'units': 'kg'},
  'food_edbl': {'value': 1995.0, 'units': 'g'},
  'atmo_co2': {'value': 0.007874524353000002, 'units': 'kg'},
  'sold_n': {'value': 0.20487643800000005, 'units': 'g'},
  'sold_p': {'value': 0.0053223559999999994, 'units': 'g'},
  'sold_k': {'value': 0.04011095900000003, 'units': 'g'},
  'enrg_kwh': {'value': 3.2673245576666674, 'units': 'kWh'},
  'h2o_tret': {'value': 4.75, 

# Run the simiulation

In [11]:
num_steps = 100
for i in range(1, num_steps + 1):
    agent_model_state = game_runner_manager.get_step(user, i)
#     if i % 100 == 0:
#         print("Step: {}".format(i))
#         pp.pprint(agent_model_state)
pp.pprint(agent_model_state)

{'agents': {'total_agent_types': {'cabbage': 10,
                                  'greenhouse_medium': 1,
                                  'human_agent': 20,
                                  'lettuce': 10,
                                  'multifiltration_purifier_post_treament': 1,
                                  'solar_pv_array': 100,
                                  'urine_recycling_processor_VCD': 1},
            'total_consumption': {'atmo_co2': {'units': 'kg',
                                               'value': 0.007874524353},
                                  'atmo_o2': {'units': 'g',
                                              'value': 984.0000000000003},
                                  'enrg_kwh': {'units': 'kWh',
                                               'value': 3.2673245576666674},
                                  'food_edbl': {'units': 'g', 'value': 1995.0},
                                  'h2o_potb': {'units': 'kg',
                                

In [12]:
game_runner_manager.save_all(allow_repeat_save=False)

In [13]:
game_runner.agent_model.get_total_storages()

{'total_currencies': {'atmo_o2': {'value': 0.0013997907155000001,
   'capacity': 400,
   'units': 'kg'},
  'atmo_co2': {'value': 199.9980793569254, 'capacity': 200, 'units': 'kg'},
  'atmo_n2': {'value': 0, 'capacity': 400, 'units': 'kg'},
  'atmo_ch4': {'value': 0, 'capacity': 200, 'units': 'kg'},
  'atmo_h2': {'value': 0, 'capacity': 100, 'units': 'kg'},
  'atmo_h2o': {'value': 200, 'capacity': 200, 'units': 'kg'},
  'food_edbl': {'value': 0.0, 'capacity': 1000, 'units': 'kg'},
  'enrg_kwh': {'value': 1000, 'capacity': 1000, 'units': 'kWh'},
  'h2o_potb': {'value': 0, 'capacity': 800, 'units': 'kg'},
  'h2o_urin': {'value': 94.36374999999926, 'capacity': 400, 'units': 'kg'},
  'h2o_wste': {'value': 236.5225000000127, 'capacity': 500, 'units': 'kg'},
  'h2o_tret': {'value': 1.36, 'capacity': 500, 'units': 'kg'},
  'sold_n': {'value': 99.97766846826366, 'capacity': 100, 'units': 'kg'},
  'sold_p': {'value': 99.99941986319257, 'capacity': 100, 'units': 'kg'},
  'sold_k': {'value': 99.99

In [14]:
game_runner.agent_model.get_total_agents()

{'total_production': {'enrg_kwh': {'value': 30.00000000000005, 'units': 'kWh'},
  'atmo_co2': {'value': 1.2800000000000007, 'units': 'kg'},
  'atmo_h2o': {'value': 2.22, 'units': 'kg'},
  'h2o_wste': {'value': 1.252499999999999, 'units': 'kg'},
  'h2o_urin': {'value': 1.25, 'units': 'kg'},
  'atmo_o2': {'value': 5.739060121999998, 'units': 'g'},
  'food_edbl': {'value': 0.0, 'units': 'g'},
  'h2o_potb': {'value': 4.75, 'units': 'kg'},
  'h2o_tret': {'value': 1.36, 'units': 'kg'}},
 'total_consumption': {'atmo_o2': {'value': 984.0000000000003, 'units': 'g'},
  'h2o_potb': {'value': 8.48862375000003, 'units': 'kg'},
  'food_edbl': {'value': 1995.0, 'units': 'g'},
  'atmo_co2': {'value': 0.007874524353, 'units': 'kg'},
  'sold_n': {'value': 0.20487643800000008, 'units': 'g'},
  'sold_p': {'value': 0.005322356, 'units': 'g'},
  'sold_k': {'value': 0.04011095900000001, 'units': 'g'},
  'enrg_kwh': {'value': 3.267324557666667, 'units': 'kWh'},
  'h2o_tret': {'value': 4.75, 'units': 'kg'},
  

In [15]:
game_runner.agent_model.get_model_stats()

{'step': 109,
 'hours_per_step': 1.0,
 'is_terminated': False,
 'time': 392400.0,
 'agents': {'total_production': {'enrg_kwh': {'value': 30.00000000000005,
    'units': 'kWh'},
   'atmo_co2': {'value': 1.2800000000000007, 'units': 'kg'},
   'atmo_h2o': {'value': 2.22, 'units': 'kg'},
   'h2o_wste': {'value': 1.252499999999999, 'units': 'kg'},
   'h2o_urin': {'value': 1.25, 'units': 'kg'},
   'atmo_o2': {'value': 5.739060121999998, 'units': 'g'},
   'food_edbl': {'value': 0.0, 'units': 'g'},
   'h2o_potb': {'value': 4.75, 'units': 'kg'},
   'h2o_tret': {'value': 1.36, 'units': 'kg'}},
  'total_consumption': {'atmo_o2': {'value': 984.0000000000003, 'units': 'g'},
   'h2o_potb': {'value': 8.48862375000003, 'units': 'kg'},
   'food_edbl': {'value': 1995.0, 'units': 'g'},
   'atmo_co2': {'value': 0.007874524353, 'units': 'kg'},
   'sold_n': {'value': 0.20487643800000008, 'units': 'g'},
   'sold_p': {'value': 0.005322356, 'units': 'g'},
   'sold_k': {'value': 0.04011095900000001, 'units': 'g

In [16]:
from simoc_server.database.db_model import AgentType

In [28]:
args, result = {}, []

agent_class, agent_name = None, None

# agent_class = 'plants'
# agent_class = 'inhabitants'
# agent_class = 'storage'

# agent_name = 'human_agent'

if agent_class:
    args["agent_class"] = agent_class
if agent_name:
    args["name"] = agent_name
    
for agent in AgentType.query.filter_by(**args).all():
    entry = {"agent_class": agent.agent_class, "name": agent.name}
    for attr in agent.agent_type_attributes:
        prefix, currency = attr.name.split('_', 1)
        if prefix not in entry:
            entry[prefix] = []
        if prefix in ['in', 'out']:
            entry[prefix].append(currency)
        else:
            entry[prefix].append({"name": currency, "value": attr.value, "units": attr.units})
    result.append(entry)
pp.pprint(result)

[{'agent_class': 'inhabitants',
  'char': [{'name': 'lifetime', 'units': 'days', 'value': '21900.0'},
           {'name': 'mass', 'units': 'kg', 'value': '60.0'},
           {'name': 'volume', 'units': 'm^3', 'value': '0.0'}],
  'in': ['atmo_o2', 'h2o_potb', 'food_edbl'],
  'name': 'human_agent',
  'out': ['atmo_co2', 'atmo_h2o', 'h2o_wste', 'h2o_urin']},
 {'agent_class': 'eclss',
  'char': [{'name': 'mass', 'units': 'kg', 'value': '0'},
           {'name': 'volume', 'units': 'm^3', 'value': '0'}],
  'in': ['h2o_wste', 'atmo_o2', 'enrg_kwh'],
  'name': 'solid_waste_aerobic_bioreactor',
  'out': ['sold_n',
          'sold_p',
          'sold_k',
          'h2o_urin',
          'atmo_co2',
          'atmo_ch4',
          'heat_cal']},
 {'agent_class': 'eclss',
  'char': [{'name': 'mass', 'units': 'kg', 'value': '0'},
           {'name': 'volume', 'units': 'm^3', 'value': '0'}],
  'in': ['h2o_tret'],
  'name': 'multifiltration_purifier_post_treament',
  'out': ['h2o_potb']},
 {'agent_clas